<a href="https://colab.research.google.com/github/IlyaZelinskiy/Wav2vec_CFT_SHIFT_LAB/blob/main/Multispeaker_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/asteroid-team/asteroid --quiet
!pip install speechbrain

# Распознавание речи нескольких говорящих

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Jan 25 07:39:02 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    25W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import torch
from torch.utils.data import DataLoader
from asteroid.data import LibriMix
import IPython.display as ipd
import numpy as np
from speechbrain.pretrained import SepformerSeparation as separator
import torchaudio
train_set, val_set = LibriMix.mini_from_download(task='sep_clean')
train_loader = DataLoader(train_set, batch_size=4, drop_last=True)
val_loader = DataLoader(val_set, batch_size=4, drop_last=True)

  0%|          | 0.00/611M [00:00<?, ?B/s]

Drop 0 utterances from 800 (shorter than 3 seconds)
Drop 0 utterances from 200 (shorter than 3 seconds)


In [15]:
train_set.df

,Unnamed: 0,mixture_ID,mixture_path,source_1_path,source_2_path,length
0,70,5400-34479-0005_4973-24515-0007,MiniLibriMix/train/mix_clean/5400-34479-0005_4...,MiniLibriMix/train/s1/5400-34479-0005_4973-245...,MiniLibriMix/train/s2/5400-34479-0005_4973-245...,121920.0
1,178,3876-174960-0020_7555-101283-0054,MiniLibriMix/train/mix_clean/3876-174960-0020_...,MiniLibriMix/train/s1/3876-174960-0020_7555-10...,MiniLibriMix/train/s2/3876-174960-0020_7555-10...,53360.0
2,207,3025-12971-0034_781-127518-0038,MiniLibriMix/train/mix_clean/3025-12971-0034_7...,MiniLibriMix/train/s1/3025-12971-0034_781-1275...,MiniLibriMix/train/s2/3025-12971-0034_781-1275...,108560.0
3,220,7285-72200-0020_4246-12545-0013,MiniLibriMix/train/mix_clean/7285-72200-0020_4...,MiniLibriMix/train/s1/7285-72200-0020_4246-125...,MiniLibriMix/train/s2/7285-72200-0020_4246-125...,103720.0
4,236,4363-14936-0060_6993-83607-0024,MiniLibriMix/train/mix_clean/4363-14936-0060_6...,MiniLibriMix/train/s1/4363-14936-0060_6993-836...,MiniLibriMix/train/s2/4363-14936-0060_6993-836...,101440.0
...,...,...,...,...,...,...
795,50547,7335-290365-0020_576-129623-0085,MiniLibriMix/train/mix_clean/7335-290365-0020_...,MiniLibriMix/train/s1/7335-290365-0020_576-129...,MiniLibriMix/train/s2/7335-290365-0020_576-129...,97840.0
796,50551,711-187466-0054_7926-254949-0057,MiniLibriMix/train/mix_clean/711-187466-0054_7...,MiniLibriMix/train/s1/711-187466-0054_7926-254...,MiniLibriMix/train/s2/711-187466-0054_7926-254...,85560.0
797,50614,7754-108640-0010_3105-166683-0032,MiniLibriMix/train/mix_clean/7754-108640-0010_...,MiniLibriMix/train/s1/7754-108640-0010_3105-16...,MiniLibriMix/train/s2/7754-108640-0010_3105-16...,87000.0
798,50719,6378-61777-0010_6006-57831-0004,MiniLibriMix/train/mix_clean/6378-61777-0010_6...,MiniLibriMix/train/s1/6378-61777-0010_6006-578...,MiniLibriMix/train/s2/6378-61777-0010_6006-578...,113680.0


In [4]:
model = separator.from_hparams(source="speechbrain/sepformer-wsj02mix", savedir='pretrained_models/sepformer-wsj02mix')

Downloading:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/113M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.3k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.2k [00:00<?, ?B/s]

In [6]:
model.device = 'cuda'

In [32]:
model.device

'cuda'

In [8]:
est_sources = model.separate_file(path='MiniLibriMix/train/mix_clean/5400-34479-0005_4973-24515-0007.wav')


In [80]:
est_sources[0,:,0].shape

torch.Size([121920])

In [61]:
est_sources[:, :, 0].shape

torch.Size([1, 121920])

In [54]:
torchaudio.save("source1hat.wav", est_sources[:, :, 0].detach().cpu(), 8000)
torchaudio.save("source2hat.wav", est_sources[:, :, 1].detach().cpu(), 8000)



In [15]:
ipd.Audio('/content/source2hat.wav', autoplay=False, rate=16000)

In [87]:
ipd.Audio('/content/source11hat.wav', autoplay=False, rate=16000)

# Перевод распознанной речи в виде фонем

In [ ]:
!pip install eng-to-ipa 
!pip install datasets==1.18.3
!pip install transformers==4.17.0

In [18]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [19]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [20]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
import torchaudio
import torch
import numpy as np

from transformers import Wav2Vec2ForCTC
mod = Wav2Vec2ForCTC.from_pretrained('/content/drive/MyDrive/diploma/n5/wav2vec_eng_to_ipa_wer(0.195)')
mod.to("cuda")

In [84]:
with torch.no_grad():
    input_values = torch.tensor(est_sources[0,:,0], device="cuda").unsqueeze(0)
    logits = mod(input_values).logits

pred_ids = torch.argmax(logits, dim=-1)
#batch["pred_str"] = processor.batch_decode(pred_ids)[0]

<ipython-input-84-20cc76fc28ee>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_values = torch.tensor(est_sources[0,:,0], device="cuda").unsqueeze(0)


In [86]:
processor.batch_decode(pred_ids)[0]

'ɪɪə ə  əɛənt  ðɛ ɪɛ  hərn ə  ə nt əɛəə ɪ  ɛhɪ wɪ ə  ɛə ˈgɛvəər  əo əv ə ˈsə'